# PMU Disturbance Analysis - Statistical Validation

Hypothesis testing, distribution fitting, and correlation analysis.

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from src import statistical, temporal, visualizations as viz
import config

sns.set_style(config.PLOT_SETTINGS['style'])
print("Libraries loaded!")

## 1. Load Data

In [ ]:
merged_df = pd.read_parquet(config.CLEANED_DATA)
pmu_df = pd.read_csv(Path(config.OUTPUT_DIR) / 'data' / 'pmu_data.csv')

datetime_cols = merged_df.select_dtypes(include=['datetime64']).columns.tolist()
datetime_col = datetime_cols[0] if datetime_cols else 'DateTime'
print(f"Data loaded. Using datetime column: {datetime_col}")

## 2. Distribution Fitting

In [ ]:
# Aggregate daily counts
daily_counts = temporal.aggregate_disturbances_by_time(merged_df, datetime_col, freq='D')

# Test distribution fit
distribution_tests = statistical.test_distribution_fit(
    daily_counts,
    distributions=['poisson', 'nbinom', 'norm']
)

print("Distribution Fitting Results:")
display(distribution_tests)

In [ ]:
# Q-Q plot for normal distribution
from scipy import stats as sp_stats

fig, ax = plt.subplots(figsize=(8, 8))
sp_stats.probplot(daily_counts, dist="norm", plot=ax)
ax.set_title('Q-Q Plot: Daily Disturbance Counts vs Normal Distribution', 
             fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
viz.save_figure(fig, '07_01_qq_plot')
plt.show()

## 3. Mann-Kendall Trend Test

In [ ]:
# Test for trend
mk_result = statistical.mann_kendall_test(daily_counts)

print("Mann-Kendall Trend Test:")
if 'error' not in mk_result:
    for key, value in mk_result.items():
        print(f"  {key}: {value}")
else:
    print(f"  Error: {mk_result['error']}")

## 4. Correlation Analysis

In [ ]:
# Select numeric columns for correlation
numeric_cols = merged_df.select_dtypes(include=[np.number]).columns.tolist()[:10]  # Limit to first 10

if len(numeric_cols) > 1:
    corr_matrix, p_values = statistical.correlation_analysis(
        merged_df, 
        columns=numeric_cols,
        method='spearman'
    )
    
    # Plot correlation heatmap
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Create significance mask
    mask = p_values > 0.05
    
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, vmin=-1, vmax=1, ax=ax,
                mask=mask, cbar_kws={'label': 'Spearman Correlation'})
    ax.set_title('Correlation Matrix (Significant Correlations Only, p<0.05)', 
                 fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    viz.save_figure(fig, '07_02_correlation_matrix')
    plt.show()
else:
    print("Insufficient numeric columns for correlation analysis")

## 5. Voltage Level vs Disturbance Test (ANOVA)

In [ ]:
# Test if voltage level affects disturbance rates
voltage_cols = [c for c in pmu_df.columns if 'voltage' in c.lower()]

if voltage_cols:
    voltage_col = voltage_cols[0]
    
    # Merge disturbance counts with PMU voltage
    disturbance_counts = merged_df.groupby('SectionID').size().reset_index(name='DisturbanceCount')
    pmu_with_counts = pmu_df.merge(disturbance_counts, on='SectionID', how='left')
    pmu_with_counts['DisturbanceCount'] = pmu_with_counts['DisturbanceCount'].fillna(0)
    
    # Perform ANOVA
    anova_result = statistical.test_voltage_disturbance_relationship(
        pmu_with_counts,
        voltage_col=voltage_col,
        disturbance_count_col='DisturbanceCount'
    )
    
    print("ANOVA: Voltage Level vs Disturbance Rate")
    if 'error' not in anova_result:
        for key, value in anova_result.items():
            print(f"  {key}: {value}")
    else:
        print(f"  Error: {anova_result['error']}")
else:
    print("No voltage column found")

## 6. Bootstrap Confidence Intervals

In [ ]:
# Calculate bootstrap CIs for key metrics
metrics = ['mean', 'median', 'std']
bootstrap_results = []

for metric in metrics:
    result = statistical.bootstrap_confidence_interval(
        daily_counts,
        statistic=metric,
        n_bootstrap=1000,
        confidence_level=config.CONFIDENCE_LEVEL
    )
    bootstrap_results.append(result)
    
print("Bootstrap Confidence Intervals (95%):")
for result in bootstrap_results:
    if 'error' not in result:
        print(f"  {result['Statistic']}: {result['Point_Estimate']:.2f} "
              f"[{result['CI_Lower']:.2f}, {result['CI_Upper']:.2f}]")

## 7. Chi-Square Independence Test

In [ ]:
# Test independence between categorical variables
cause_cols = [c for c in merged_df.columns if 'cause' in c.lower()]
type_cols = [c for c in merged_df.columns if 'type' in c.lower()]

if cause_cols and type_cols:
    chi2_result = statistical.chi_square_independence_test(
        merged_df,
        col1=cause_cols[0],
        col2=type_cols[0]
    )
    
    print("Chi-Square Independence Test:")
    print(f"  {chi2_result['Interpretation']}")
    print(f"  Chi2 Statistic: {chi2_result['Chi2_Statistic']:.2f}")
    print(f"  P-value: {chi2_result['P_Value']:.4f}")

## 8. Save Results

In [ ]:
# Compile statistical validation results
validation_results = pd.DataFrame({
    'Test': ['Distribution Fit (Best)', 'Mann-Kendall Trend', 'Bootstrap CI Mean'],
    'Result': [
        distribution_tests.iloc[0]['Distribution'] if len(distribution_tests) > 0 else 'N/A',
        mk_result.get('Trend', 'Error'),
        f"{bootstrap_results[0]['Point_Estimate']:.2f} [{bootstrap_results[0]['CI_Lower']:.2f}, {bootstrap_results[0]['CI_Upper']:.2f}]" if bootstrap_results else 'N/A'
    ]
})

validation_results.to_csv(config.STATISTICAL_RESULTS, index=False)
print(f"\nStatistical validation results saved to: {config.STATISTICAL_RESULTS}")
display(validation_results)

## Summary

- ✅ Tested distribution fits (Poisson, Negative Binomial, Normal)
- ✅ Performed Mann-Kendall trend test
- ✅ Calculated correlation matrix with significance testing
- ✅ Tested voltage-disturbance relationship (ANOVA)
- ✅ Generated bootstrap confidence intervals
- ✅ Performed chi-square independence tests

**Analysis Complete!** All 7 notebooks executed.